## Tangent - automatic differentiation library

If you've ever had to write down the differentiations manually, you know how painful and slow it can be.

I found [this page](http://cs231n.github.io/optimization-2/) from CS231n stanford course really useful when I started learning about neural networks.

<img src="https://raw.githubusercontent.com/google/tangent/master/docs/sct-ad-tf.gif">

[Tangent](https://github.com/google/tangent) isn't the first such library, but it's certainly very easy to use. It uses the Abstract Syntax Tree (AST) of the function. Once it has the AST, it can simply walk in the reverse order to compute the gradient.

### Usage
It's very easy to use:
```python
import tangent
def f(x):
    a = x * x
    b = x
    return a + b
df = tangent.grad(f)
```

`df` in the above example is derivative wrt x. By default, derivative wrt the first argument is computed.


### Pros
* **Supports native python operations**: This makes it very easy to use compared to [Tensorflow](https://www.tensorflow.org/) or [Pytorch](http://pytorch.org/).
* **Supports numpy/tensorflow-eager functions**: Gradient operations for the tensorflow-eager/numpy are built-in.
* **Higher Order Gradients**: You can easily do higher order gradients by simply taking a `grad` of a `grad`.

### Cons
* Doesn't support classes yet.

In [1]:
import tangent
import numpy as np

/Users/hardik/h/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Reverse mode

In [2]:
def f(x, y):
    return x * x + y * y

dfdx = tangent.grad(f, wrt=(0,), mode='reverse', verbose=True)
dfdy = tangent.grad(f, wrt=(1,), mode='reverse', verbose=False)
dfdxy = tangent.grad(f, wrt=(0, 1), mode='reverse', verbose=False)

def dfdx(x, y, b_return=1.0):
    x_times_x = x * x

    # Grad of: _return = x_times_x + y_times_y
    _bx_times_x = tangent.unbroadcast(b_return, x_times_x)
    bx_times_x = _bx_times_x

    # Grad of: x_times_x = x * x
    _bx = tangent.unbroadcast(bx_times_x * x, x)
    _bx2 = tangent.unbroadcast(bx_times_x * x, x)
    bx = _bx
    bx = tangent.add_grad(bx, _bx2)
    return bx



We know `dfdx` is `2 * x` and `dfdy` is `2 * y`.

In [3]:
print('f(x, y):', f(4, 4))
print('dfdx:', dfdx(4, 4))
print('dfdy:', dfdx(4, 4))
print('dfdxy:', dfdxy(4, 4))

f(x, y): 32
dfdx: 8.0
dfdy: 8.0
dfdxy: (8.0, 8.0)


### Conditionals
In this example, we will see how Tangent handles if/else statements.

In [4]:
def f(x, y):
    if x > 2:
        z = 2 * x * y
    else:
        z = x * x + 3 * y * y
    return z

dfdy = tangent.grad(f, wrt=(1,), mode='reverse', verbose=True)

def dfdy(x, y, bz=1.0):
    # Initialize the tape
    _stack = tangent.Stack()
    _z = None
    _3_times_y = None
    _2_times_x = None

    # Beginning of forward pass
    cond = x > 2
    if cond:
        _2_times_x = 2 * x
    else:
        _3_times_y = 3 * y
        _z = _3_times_y * y
    tangent.push(_stack, cond, '_236eb625')

    # Beginning of backward pass
    cond = tangent.pop(_stack, '_236eb625')
    if cond:
        # Grad of: z = 2 * x * y
        _by = tangent.unbroadcast(bz * _2_times_x, y)
        by = _by
    else:
        # Grad of: z = x * x + 3 * y * y
        _b_z = tangent.unbroadcast(bz, _z)
        b_z = _b_z
        _b_3_times_y = tangent.unbroadcast(b_z * y, _3_times_y)
        _by3 = tangent.unbroadcast(b_z * _3_times_y, y)
        b_3_times_y = _b_3_times_y
        by = _by3
        _by2 = tangent.unbroadcast(b_3_times_y * 3, y)
        by = tangent.add_grad(by, _by2)
    return by



In [5]:
print('dfdy(4,4) (Expected: 2 * x):', dfdy(4, 4))
print('dfdy(1,4) (Expected: 3 * 2 * y):', dfdy(1, 4))

dfdy(4,4) (Expected: 2 * x): 8.0
dfdy(1,4) (Expected: 3 * 2 * y): 8.0


Well, it fails. There seems to be a bunch of bugs like this. For example, if I don't assign `z` and directly return from the if/else, it fails.